We will analyze the Uber Pickups in New York City dataset. This is more of a data visualization project

https://data-flair.training/blogs/r-data-science-project-uber-data-analysis/

Data source:

https://www.kaggle.com/fivethirtyeight/uber-pickups-in-new-york-city/data    

In [1]:
%matplotlib inline

# pretty display of variables
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [28]:
#!dir "./data/uber-pickups-in-new-york-city"

In [4]:
import pandas as pd

In [16]:
apr_data = pd.read_csv("./data/uber-pickups-in-new-york-city/uber-raw-data-apr14.csv")
may_data = pd.read_csv("./data/uber-pickups-in-new-york-city/uber-raw-data-may14.csv")
jun_data = pd.read_csv("./data/uber-pickups-in-new-york-city/uber-raw-data-jun14.csv")
jul_data = pd.read_csv("./data/uber-pickups-in-new-york-city/uber-raw-data-jul14.csv")
aug_data = pd.read_csv("./data/uber-pickups-in-new-york-city/uber-raw-data-aug14.csv")
sep_data = pd.read_csv("./data/uber-pickups-in-new-york-city/uber-raw-data-sep14.csv")


In [17]:
frames = [apr_data, may_data, jun_data, jul_data, aug_data, sep_data]

data_2014  = pd.concat(frames)

In [18]:
data_2014.head()

,Date/Time,Lat,Lon,Base
0,4/1/2014 0:11:00,40.7690,-73.9549,B02512
1,4/1/2014 0:17:00,40.7267,-74.0345,B02512
2,4/1/2014 0:21:00,40.7316,-73.9873,B02512
3,4/1/2014 0:28:00,40.7588,-73.9776,B02512
4,4/1/2014 0:33:00,40.7594,-73.9722,B02512


In [21]:
data_2014.shape

(4534327, 4)

In [31]:
data_2014.Base.value_counts()

B02617    1458853
B02598    1393113
B02682    1212789
B02764     263899
B02512     205673
Name: Base, dtype: int64